# Building an Encoder Tranformer Architecture

Attention mechanism: capture long-range dependencies. Use whole sequence to weight each element.
Create a vector describing the token's position in the sequence <- transformers process tokens simultaneously

In [1]:
import torch
import torch.nn as nn

#  Create a vector describing the token's position in the sequence
class PositionalEncoder(nn.Module):
    def __init__(self, d_model, max_seq_length=512):
        super(PositionalEncoder, self).__init__()
        
        # set max length and embedding size
        self.d_model = d_model
        self.max_seq_length = max_seq_length
        
        # create matrix for sequences up to max_seq_length
        pe = torch.zeros(max_seq_length, d_model)

        # position indices in the sequence
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)

        # scale position indices
        div_term = torch.exp(torch.arange(0, d_model, 2, dtype=torch.float) * -(math.log(10000.0) / d_model))
        # alternate sin and cos to assign unique position encodings to the matrix pe
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)

        # set pe as non-trainable
        self.register_buffer('pe', pe)

    def forward(self, x):
        """Add positional encodings pe to input tensor of sequence embeddings, X"""
        x = x + self.pe[:, :x.size(1)]
        return x

Self-attention: understand the interrelationship between words in a sequence.

1. Input: sequence of embeddings.
2. Project the sequence of embeddings into 3 sequences of same dimensions: Q:query, K:key, V:values, by applying 3 linear transformations, each with its own weights learned during training
3. Compute a Q-K similarity matrix of attention scores (scaled dot-product self-attention)
4. Apply softmax to get a matrix of attention weights

In practice, transformers paralellize attention heads

In [2]:
import torch.nn as nn
import torch.nn.functional as F

class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        """
        num_heads: number of attention heads, each handling embeddings of size head_dim
        """
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        self.head_dim = d_model // num_heads

        # define 3 linear transformatins for attention input
        self.query_linear = nn.Linear(d_model, d_model)
        self.key_linear = nn.Linear(d_model, d_model)
        self.value_linear = nn.Linear(d_model, d_model)

        # define linear transformation for the final concatenated output
        self.output_linear = nn.Linear(d_model, d_model)

    def split_heads(self, x, batch_size):
        """Split the inputs accross attention heads"""
        x = x.view(batch_size, -1, self.num_heads, self.head_dim)
        return x.permute(0, 2, 1, 3).contiguous().view(batch_size * self.num_heads, -1, self.head_dim)

    def compute_attention(self, query, key, mask=None):
        """Computes attention weights inside each head."""
        # Compute dot-product attention scores
        scores = torch.matmul(query, key.permute(1, 2, 0))
        if mask is not None:
            scores = scores.masked_fill(mask == 0, float("-1e9"))
        # Normalize attention scores into attention weights
        attention_weights = F.softmax(scores, dim=-1)
        return attention_weights

    def forward(self, query, key, value, mask=None):
        batch_size = query.size(0)

        # attention weights for Q, K, V
        query = self.split_heads(self.query_linear(query), batch_size)
        key = self.split_heads(self.key_linear(key), batch_size)
        value = self.split_heads(self.value_linear(value), batch_size)

        # concatenate
        attention_weights = self.compute_attention(query, key, mask)

        # Multiply attention weights by values, concatenate and linearly project outputs
        output = torch.matmul(attention_weights, value)
        output = output.view(batch_size, self.num_heads, -1, self.head_dim).permute(0, 2, 1, 3).contiguous().view(batch_size, -1, self.d_model)
        return self.output_linear(output)

## Building an encoder transformer
An encoder-based transformer that incorporates the self-atttention mechanism above.

The original transformer architecture incorporates an encoder and a decoder, for sequence-to-sequence language generation.

A simpler form only includes an encoder, for text classification, named-entity, intent recognition...

Tranformer body
> Stack of encoder layers. Each layer has multi-headed self-attention. Followed by feed-forward layer. Also includes layers required by training: dropout, normalization...

Transformer head
> Final layer, produces task-specific output. In encoder-only transformer: typical supervised learning tasks such as classification, regression

#### Feed-forward layer
2 x fully connected + ReLU activation

d_ff: dimension used between linear layers, typically different from the d_model dimension

In [3]:
import torch.nn as nn

class FeedForwardSubLayer(nn.Module):
    # Specify the two linear layers' input and output sizes
    def __init__(self, d_model, d_ff):
        super(FeedForwardSubLayer, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

	# Apply a forward pass
    def forward(self, x):
        return self.fc2(self.relu(self.fc1()))

#### Encoder layer
* Multi-headed self-attention
* Feed-forward sublayer
* Layer normalization and dropout

In forward-pass, uses a mask to prevent processing of padding tokens

In [4]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = FeedForwardSubLayer(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        return self.norm2(x + self.dropout(ff_output))

In [5]:
from torch import nn, Tensor
import math

class PositionalEncoding(nn.Module):

    def __init__(self, d_model: int, dropout: float = 0.1, max_len: int = 5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)

        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-math.log(10000.0) / d_model))
        pe = torch.zeros(max_len, 1, d_model)
        pe[:, 0, 0::2] = torch.sin(position * div_term)
        pe[:, 0, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x: Tensor) -> Tensor:
        """
        Arguments:
            x: Tensor, shape ``[seq_len, batch_size, embedding_dim]``
        """
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

#### The encoder
* Input embeddings based on vocab_size
* Positional encoding
* Stack of encoder layers

In [6]:
class TransformerEncoder(nn.Module):
    def __init__(self, vocab_size, d_model, num_layers, num_heads, d_ff, dropout, max_sequence_length):
        super(TransformerEncoder, self).__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.positional_encoding = PositionalEncoder(d_model, max_sequence_length)
        # Define a stack of multiple encoder layers
        self.layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
	
    # Complete the forward pass method
    def forward(self, x, mask):
        x = self.embedding(x)
        x = self.positional_encoding(x)
        for layer in self.layers:
            x = layer(x, mask)
        return x

#### Tranformer head for classification
Application: classification, sentiment, extractive QA

Transforms hidden states into num_classes probabilities

In [7]:
class ClassifierHead(nn.Module):
    def __init__(self, d_model, num_classes):
        super(ClassifierHead, self).__init__()
        # Add linear layer for multiple-class classification
        self.fc = nn.Linear(d_model, num_classes)

    def forward(self, x):
        logits = self.fc(x[:, 0, :])
        # Obtain log class probabilities upon raw outputs
        return F.log_softmax(logits, dim=-1)

#### Transformer head for regression
Application: estimate readability, measure complexity,...

output_dim is 1 when predicting 1 value

In [8]:
class RegressionHead(nn.Module):
    def __init__(self, d_model, output_dim):
        super(RegressionHead, self).__init__()
        self.fc = nn.Linear(d_model, output_dim)

    def forward(self, x):
        return self.fc(x)

### Testing 
Testing the encoder transformer with random data

In [9]:
num_classes = 3
vocab_size = 10000
batch_size = 8
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
sequence_length = 256
dropout = 0.1

(following code might not be correct)

In [10]:
input_sequence = torch.randint(0, vocab_size, (batch_size, sequence_length))
mask = torch.randint(0, 2, (sequence_length, sequence_length))

# Instantiate the encoder transformer's body and head
encoder = TransformerEncoder(vocab_size, d_model, num_layers, num_heads, d_ff, dropout, max_sequence_length=sequence_length)
classifier = ClassifierHead(d_model, num_classes)

# Complete the forward pass 
output = encoder(input_sequence, mask)
classification = classifier(output)
print("Classification outputs for a batch of ", batch_size, "sequences:")
print(classification)

RuntimeError: The size of tensor a (64) must match the size of tensor b (256) at non-singleton dimension 0

In [ ]:
len(classification[0][0])